In [1]:
import ee
import colorcet
from matplotlib.cm import get_cmap
import geemap

ee.Initialize(project='hotspotstoplight')

In [2]:
URL = 'gs://hotspotstoplight_floodmapping/data/nicaragua/outputs/flood_prob.tif'
image = geemap.load_GeoTIFF(URL)

In [3]:
jrc = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality')
water = jrc.select('seasonality').gte(10)
image = image.where(water, 0)

In [4]:
vizParamsProbability = {
    'min': 0.6,
    'max': 1,
    'palette': colorcet.bmw
}

viz_params_pop = {
    'min': 0,
    'max': 1,
    'palette': colorcet.gwv
}

viz_params_vuln = {
    'min': -7,
    'max': 17,
    'palette': colorcet.bmy
}

elevationPalette = ['006600', '002200', 'fff700', 'ab7634', 'c4d0ff', 'ffffff']
visParams = {'min': 1, 'max': 3000, 'palette': elevationPalette}

Map = geemap.Map()
Map.add("layer_manager")
Map.add("basemap_selector")

geometry = image.geometry()
Map.centerObject(geometry, 8)
Map.addLayer(image, vizParamsProbability, 'Flood Probability')
# Map.addLayer(exp, viz_params_pop, "Exposure Data")
# Map.addLayer(vuln, viz_params_vuln, "Vulnerability Data")
# Map.addLayer(fabdem, visParams, "FABDEM")
Map

Map(center=[12.922089987955754, -85.22913405567712], controls=(WidgetControl(options=['position', 'transparent…